## Convention Database

In this notebook we store our convention data in a database. See the README for details.

In [5]:
import re
import sqlite3
import os
from collections import defaultdict
cwd=os.getcwd()
cwd

'/Users/steviodong/Desktop/Text Mining/exercise-conventions-to-db-StevioD'

In [10]:
# Need your path here
path_to_files = '/Users/steviodong/Desktop/Text Mining/exercise-conventions-to-db-StevioD/' 
transcript_files = os.listdir(path_to_files)

In [11]:
transcript_files

['.DS_Store',
 'Convention DB - Solution.ipynb',
 'README.md',
 '.gitignore',
 '2020_Conventions.db',
 'www_rev_com_blog_transcripts2020-democratic-national-convention-dnc-night-4-transcript.txt',
 '.ipynb_checkpoints',
 '.git',
 'Convention DB.ipynb']

Let's start by creating a lookup between the files and the nights/parties. 

In [14]:
# Just going to be a two-item list with party and night.
lookup_party_night = defaultdict(list)

# Do some reading about default dictionaries (`defaultdict`)
# Ask me about them in class. They're a bit confusing, but
# really powerful. 

file_night = re.compile(r"night-[1-4]")

for file in transcript_files :
    if "rnc" in file :
        lookup_party_night[file].append("Republican")
    elif "dnc" in file : 
        lookup_party_night[file].append("Democratic")
        
    night_text = file_night.search(file).group(0)    
    lookup_party_night[file].append(night_text.split("-")[1])


AttributeError: 'NoneType' object has no attribute 'group'

Now let's set up our DB.

In [16]:
db = sqlite3.connect("2020_Conventions.db")
cur = db.cursor()

In [17]:
# Build an empty table called `conventions` 
# with the structure described
# in the README. 


db.commit()

Okay, that was easy enough. Now the tough part. We're going to need to use a regular expression to match the speaker, split the text on that, and funnel everything into the right spot. We wrapped the text, so we'll clean returns out of the text as we put it in. 

In [ ]:
# Fill in a regex that matches speakers and the subsequent time.
# Use one from class or make your own!

speaker_time_pattern = re.compile(r" YOUR REGEX HERE   ")

### Note
In my solution I needed these two functions. The first one you may not need or you may need a different version of. The second one I've left gaps for you to fill in.

Both of these functions are dependent on the regex you end up using. If you use the same one we did in class, which is the one I think is in my solutions, then you can just follow along. If you use a different regex, you should rework the second function, at least. 

In [ ]:
def fix_speakers(speaker) : 
    # Missing punctuation causes our process to fail. We could engineer a solution, 
    # by doing multiple passes and removing speakers from text, but that could
    # be fraught too. We'll just fix things manually and not worry about the 
    # textual errors. 
    # 
    # One issue is there are a handful of cases where the closing puctuation is a
    # dash. This also occurs in hyphenated names, so the fix to the regex would 
    # be kind of a hassle. Seemed easier to do the lookup.
    # 
    # Note this pretty handy way to do these fixes. 

    fix_lu = {   
        'Will we see to it that no one who works full time can live in poverty Pete Buttigieg':'Pete Buttigieg',
        'I stayed for an hour and a half- Jamie Ponder':'Jamie Ponder',
        'It seems like just yesterday that we were at our first convention- Melania Trump':'Melania Trump',
        'The night before I fought back- Kayleigh McEnany':'Kayleigh McEnany',
        'It is a sad irony that Jackie immigrated- Speaker 8':'Speaker 8',
        'I withdrew from the terrible one- Donald Trump':'Donald Trump',
        'Alaska casts seven votes for Bernie- Chuck Degnan':'Chuck Degnan',
        'A lot of us were shocked and I think what gives me hope- Art Acevedo':'Art Acevedo',
        'But Joe Biden is a guy who has earned the respect- Eva Longoria':'Eva Longoria',
        'And the promise of our country led by president Joe Biden and vice-president Kamala Harris Kerry Washington':'Kerry Washington',
        'Joe will also- Senator Bernie Sanders':'Senator Bernie Sanders'
    }

    if speaker in fix_lu : 
        return(fix_lu[speaker])
    else : 
        return(speaker)
    

In [ ]:
def parse_speakers_times(st_items) :
    # Since we'll need to split these carefully, let's
    # write a function to just split one of them. They come in 
    # like '. Name Name: ( HH:MM:SS )' with unknown first punctuation
    # And complicated names. We'll return three lists in a dictionary
    # with keys "speakers", "times", "last_puncts" holding the 
    # speakers, the times, and the closing punctuation, since 
    # that needs to be glued onto the text. 
    
    # The time fields come in looking like this ' (NN' or 
    # 'NN )'. We need a regular expression that matches 
    # the characters that *aren't* NN (which stands 
    # for the two digit hour or seconds).
    time_clean = re.compile(r" YOUR REGEX HERE")
    
    speakers = []
    times = []
    last_puncts = []
    
    # There are probably some more efficient ways to do 
    # what I'm doing in this loop. 
    for st_item in st_items : 
        last_puncts.append(st_item[0])
        
        # split the pieces on semi-colon
        pieces = st_item[1:].split(":")
        
        # Figure out what to append to speakers.
        this_speaker = "Something you figure out. `pieces` seems useful."
        speakers.append(this_speaker)
        
        # This one took a while to get right
        # Turn the remaining pieces into a clean time.
        this_time = "01:23" # always this time, till you figure out how to get
        # the right one in there. 

        times.append(this_time)
     
    # If you do a manual lookup to do some fixes,
    # like I do, uncomment this next line.
    #speakers = list(map(fix_speakers,speakers))
        
    assert(len(speakers)==len(times))
    assert(len(speakers)==len(last_puncts))
        
    return({'speakers':speakers,
            'times':times,
            'last_puncts':last_puncts})

In [ ]:
for file in transcript_files : 
    with open(path_to_files + file,encoding="UTF-8") as infile :
        file_text = infile.read()

        # This command will give us all the matches
        speakers_times = speaker_time_pattern.findall(file_text)

        # And this one will split the file
        split_texts = speaker_time_pattern.split(file_text)

        # First bit of text is unattributed. Keeping the
        # formatting the same

        speakers_times = ['. Unknown: ( 00:00 )'] + speakers_times

        # Do the parsing in a function
        results = parse_speakers_times(speakers_times)

        # The last mark of punctuation from text N was 
        # caught in the Speaker in the N+1 position. let's fix that. 
        texts = [split_texts[idx] + results['last_puncts'][idx] for idx in range(len(split_texts))]

        # A little cleaning of the texts
        texts = [t.strip().replace("\n"," ") for t in texts]

        # All we need now is a counter to keep track of how many times this speaker has 
        # spoken on this night. 
        speaker_counter = defaultdict(int)

        assert(len(results['speakers'])==len(texts))
        
        for idx in range(len(texts)) : 
            
            # build a row to put in the database
            db_row = ('you need',
                      'to figure',
                      'out how',
                       2,
                      'fill in all',
                       8,
                      'of these fields.')
   
            cur.execute('''INSERT INTO conventions
                                (party,night,speaker,speaker_count,
                                time,text,text_len,file) 
                           VALUES (?,?,?,?,?,?,?,?)''',
                       db_row) 

    # Might as well commit each file
    db.commit()

In [ ]:
# Let's run some queries and see how things look
for row in cur.execute('''
        SELECT party, sum(text_len) as words
        FROM conventions
        GROUP BY party
        ORDER BY words DESC
''') : 
    print(f"The {row[0]} party had {row[1]} words.")

In [ ]:
for row in cur.execute('''
        SELECT party, speaker, sum(text_len) as words
        FROM conventions
        GROUP BY party, speaker
        ORDER BY words DESC
        LIMIT 50
''') : 
    print(f"For the {row[0]} party, {row[1]} said {row[2]} words.")

In [ ]:
for row in cur.execute('''
        SELECT party, night, sum(text_len) as words
        FROM conventions
        GROUP BY party, night
        ORDER BY words DESC
''') : 
    print(f"The {row[0]} party on night {row[1]} said {row[2]} words.")

In [ ]:
db.close()